
# Python en los negocios: Estadística descriptiva con una billetera móvil (15 min)

**Objetivo:** En 15 minutos veremos cómo usar Python para responder preguntas de negocio con **estadística descriptiva** aplicada a una billetera móvil.

**KPIs mensuales que mediremos:**
- **Facturación (GMV):** suma de montos exitosos.
- **Usuarios Activos (MAU):** usuarios con ≥1 transacción exitosa en el mes.
- **Ticket Promedio (AOV):** GMV / número de transacciones exitosas.
- **# Transacciones:** conteo de transacciones exitosas.

> Dataset sintético incluido: `transactions_wallet_sample.csv`


In [ ]:

# === 1) Setup e importaciones ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.float_format", lambda v: f"{v:,.2f}")


In [ ]:

# === 2) Cargar datos (usa el CSV adjunto o súbelo en Colab) ===
# Si estás en Google Colab, ejecuta:
# from google.colab import files
# uploaded = files.upload()  # y sube transactions_wallet_sample.csv

df = pd.read_csv("transactions_wallet_sample.csv", parse_dates=["txn_datetime"])
df.head()



### Esquema del dataset
- `txn_id`: identificador único de transacción
- `txn_datetime`: fecha y hora
- `month`: periodo YYYY-MM
- `user_id`: identificador del usuario
- `merchant_id`, `merchant_name`, `category`: información del comercio
- `payment_method`: método de pago usado
- `amount_usd`: monto de la transacción
- `cashback_usd`: cashback otorgado
- `status`: `success`, `reversed` o `failed` (solo **success** cuenta para KPIs)


In [ ]:

# === 3) Limpieza básica: nos quedamos con transacciones exitosas ===
df_success = df.query("status == 'success'").copy()

# Asegurar tipos útiles
df_success["month"] = pd.PeriodIndex(df_success["month"], freq="M").astype(str)
df_success["date"] = df_success["txn_datetime"].dt.date

print("Transacciones totales:", len(df))
print("Transacciones exitosas:", len(df_success))
df_success.head(3)


In [ ]:

# === 4) Funciones helper para KPIs mensuales ===
def kpis_mensuales(data):
    # GMV por mes
    gmv = data.groupby("month")["amount_usd"].sum().rename("GMV_usd")
    # Transacciones por mes
    txns = data.groupby("month")["txn_id"].count().rename("transactions")
    # Usuarios activos por mes (usuarios con >=1 txn)
    mau = data.groupby("month")["user_id"].nunique().rename("active_users")
    # Ticket promedio (AOV)
    aov = (gmv / txns).rename("avg_ticket_usd")

    out = pd.concat([gmv, txns, mau, aov], axis=1).reset_index()
    return out.sort_values("month")

kpis = kpis_mensuales(df_success)
kpis



### Visualización rápida de KPIs
> Nota: Usamos **matplotlib** y cada gráfico en una figura separada.


In [ ]:

# === 5) Gráfico: GMV mensual ===
plt.figure()
plt.plot(kpis["month"], kpis["GMV_usd"], marker="o")
plt.title("GMV mensual (USD)")
plt.xlabel("Mes")
plt.ylabel("GMV (USD)")
plt.xticks(rotation=45)
plt.grid(True)
plt.show()


In [ ]:

# === 6) Gráfico: Usuarios activos mensuales ===
plt.figure()
plt.plot(kpis["month"], kpis["active_users"], marker="o")
plt.title("Usuarios activos mensuales (MAU)")
plt.xlabel("Mes")
plt.ylabel("# Usuarios activos")
plt.xticks(rotation=45)
plt.grid(True)
plt.show()


In [ ]:

# === 7) Gráfico: Ticket promedio mensual ===
plt.figure()
plt.plot(kpis["month"], kpis["avg_ticket_usd"], marker="o")
plt.title("Ticket promedio mensual (AOV)")
plt.xlabel("Mes")
plt.ylabel("USD por transacción")
plt.xticks(rotation=45)
plt.grid(True)
plt.show()


In [ ]:

# === 8) Gráfico: Número de transacciones mensuales ===
plt.figure()
plt.plot(kpis["month"], kpis["transactions"], marker="o")
plt.title("Transacciones mensuales")
plt.xlabel("Mes")
plt.ylabel("# Transacciones")
plt.xticks(rotation=45)
plt.grid(True)
plt.show()



## Estadística descriptiva útil para negocio


In [ ]:

# === 9) Descriptivos generales del monto ===
desc = df_success["amount_usd"].describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95]).to_frame("amount_usd")
desc


In [ ]:

# === 10) Distribución del ticket (histograma) ===
plt.figure()
plt.hist(df_success["amount_usd"], bins=30)
plt.title("Distribución de montos por transacción")
plt.xlabel("Monto (USD)")
plt.ylabel("Frecuencia")
plt.grid(True)
plt.show()



### ¿Quiénes impulsan el negocio? Comercios y categorías


In [ ]:

# === 11) Top 5 comercios por GMV ===
top_merchants = (df_success.groupby("merchant_name")["amount_usd"]
                 .sum().sort_values(ascending=False).head(5).reset_index())
top_merchants


In [ ]:

# === 12) GMV por categoría ===
gmv_cat = df_success.groupby("category")["amount_usd"].sum().sort_values(ascending=False).reset_index()
gmv_cat


In [ ]:

# === 13) Método de pago más usado (% de transacciones) ===
method_share = (df_success.groupby("payment_method")["txn_id"].count()
                .div(len(df_success)).mul(100).round(2).rename("share_%").reset_index())
method_share



## Resumen ejecutivo mensual


In [ ]:

# === 14) Tabla de KPIs con % cambio mes a mes ===
kpis2 = kpis.copy()
kpis2[["GMV_usd_mom_%","transactions_mom_%","active_users_mom_%","avg_ticket_mom_%"]] = (
    kpis2[["GMV_usd","transactions","active_users","avg_ticket_usd"]]
    .pct_change() * 100
)
kpis2



### Bonus (opcional si hay tiempo): Nuevos vs. recurrentes


In [ ]:

# === 15) Nuevos vs recurrentes: un vistazo rápido ===
first_txn_month = (df_success.sort_values("txn_datetime")
                   .groupby("user_id")["month"].first().rename("first_month"))
tmp = df_success.merge(first_txn_month, on="user_id", how="left")
tmp["user_type"] = np.where(tmp["month"] == tmp["first_month"], "new", "returning")

cohort = (tmp.groupby(["month","user_type"])["user_id"]
          .nunique().unstack(fill_value=0).reset_index())
cohort


In [ ]:

# Gráfico simple: nuevos vs recurrentes por mes
plt.figure()
plt.plot(cohort["month"], cohort["new"], marker="o", label="Nuevos")
plt.plot(cohort["month"], cohort["returning"], marker="o", label="Recurrentes")
plt.title("Usuarios activos: nuevos vs recurrentes")
plt.xlabel("Mes")
plt.ylabel("# Usuarios")
plt.xticks(rotation=45)
plt.grid(True)
plt.legend()
plt.show()



---

## Guion sugerido (15 minutos)
1. **Contexto (2 min):** rol de la analítica descriptiva para entender el desempeño mensual.
2. **Carga & limpieza (4 min):** filtrar exitosas; validar campos clave.
3. **KPIs (5 min):** GMV, MAU, AOV, transacciones + tendencias con gráficos.
4. **Descriptivos (2 min):** distribución de tickets, top comercios/métodos.
5. **Cierre (2 min):** decisiones accionables (p.ej., enfocarse en categorías/aliados que traccionan, revisar meses con caída de AOV).

> Tarea: replicar con tus datos reales y añadir métricas propias (retención, churn, cashback/gasto, conversión por método de pago).
